In [1]:
import tensorflow as tf

C:\Users\13936\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### 创建和使用张量

In [2]:
# 张量是一个多维数组。 与NumPy ndarray对象相似，tf.Tensor对象具有数据类型和形状。
# 此外，tf.Tensor可以驻留在加速器内存（如GPU）中。
# TensorFlow提供了丰富的操作库（tf.add，tf.matmul，tf.linalg.inv等），
# 这些操作消耗并产生了tf.Tensors。
# 这些操作会自动转换本机Python类型，例如：

print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))
print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


In [ ]:
# NumPy数组和tf.Tensor之间最明显的区别是：
# 张量可以由加速器内存（例如GPU，TPU）支持。
# 张量是不变的。

# NumPy兼容性
# 在TensorFlow tf.Tensors和NumPy ndarray之间进行转换很容易：
# TensorFlow操作会自动将NumPy ndarray转换为Tensors。
# NumPy操作会自动将张量转换为NumPy ndarray。
# 使用它们的.numpy（）方法将张量显式转换为NumPy ndarray。
# 这些转换通常很便宜，因为如果可能的话，数组和tf.Tensor共享底层的内存表示形式。 
# 但是，共享底层表示并不总是可能的，因为tf.Tensor可能托管在GPU内存中，而NumPy数组始终由主机内存支持，并且转换涉及从GPU到主机内存的复制。

In [4]:
import numpy as np

ndarray = np.ones([3, 3])

print("Tensorflow操作将numpy数组转变为Tensor")
tensor = tf.multiply(ndarray, 42)
print(tensor)

print("numpy操作将Tensor转变为numpy数组")
print(np.add(tensor, 1))

print(".numpy() 将Tensor转变为numpy数组")
print(tensor.numpy())

Tensorflow操作将numpy数组转变为Tensor
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
numpy操作将Tensor转变为numpy数组
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
.numpy() 将Tensor转变为numpy数组
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


In [6]:
# 使用GPU进行计算可加速许多TensorFlow操作。

x = tf.random.uniform([3, 3])

print("是否有可用的GPU: "),
print(tf.config.experimental.list_physical_devices("GPU"))

print("张量是否在GPU上:  "),
print(x.device.endswith('GPU:0'))

是否有可用的GPU: 
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
张量是否在GPU上:  
True


### 使用GPU加速

In [2]:
import time

def time_matmul(x):
    start = time.time()
    for loop in range(10):
        tf.matmul(x, x)

    result = time.time()-start

    print("10 loops: {:0.2f}ms".format(1000*result))

# 强制在CPU上执行
print("On CPU:")
with tf.device("CPU:0"):
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("CPU:0")
    time_matmul(x)

# 强制在GPU上执行（如果有的话）
if tf.config.experimental.list_physical_devices("GPU"):
    print("On GPU:")
    with tf.device("GPU:0"): # Or GPU:1 for the 2nd GPU, GPU:2 for the 3rd etc.
        x = tf.random.uniform([1000, 1000])
        assert x.device.endswith("GPU:0")
        time_matmul(x)

On CPU:
10 loops: 64.82ms
On GPU:
10 loops: 263.30ms


### 演示 tf.data.Dataset

In [3]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# Create a CSV file
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
    f.write("""Line 1
Line 2
Line 3
  """)

ds_file = tf.data.TextLineDataset(filename)

In [4]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

In [9]:
print('ds_tensors:')
for x in ds_tensors:
    print(x)

print('\nds_file:')
for x in ds_file:
    print(x)

ds_tensors:
tf.Tensor([1 4], shape=(2,), dtype=int32)
tf.Tensor([ 9 16], shape=(2,), dtype=int32)
tf.Tensor([25 36], shape=(2,), dtype=int32)

ds_file:
tf.Tensor([b'Line 1' b'Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'Line 3' b'  '], shape=(2,), dtype=string)
